In [1]:
import math
import numpy as np

In [ ]:
# Link lengths and shoulder height (same as in forward_kinematics_points)
L1 = 6.1   # Shoulder → elbow
L2 = 7.0   # Elbow → end effector
H  = 6.8   # Shoulder height above ground

def inverse_kinematics_point(x, y, z, elbow_up=False): # ta ao contrario o booleano
    """
    Compute inverse kinematics for the 3-DOF arm.

    Args:
        x, y, z: Desired end-effector coordinates.
        elbow_up: If True, returns the "elbow-up" solution; otherwise "elbow-down".

    Returns:
        (theta1_deg, theta2_deg, theta3_deg): Joint angles in degrees.
    
    Raises:
        ValueError: If the point is out of reach.
    """
    # 1) XY-plane yaw:
    #    φ = θ1_rad
    phi = math.atan2(y, x)
    #    Map back to user angle:
    theta1_deg = math.degrees(phi + math.pi/2)

    # 2) Project into the shoulder's pitch plane:
    #    Horizontal distance from shoulder:
    r = math.hypot(x, y)
    #    Vertical offset from shoulder:
    dz = z - H

    # 3) Two‑link planar IK:
    D = (r*r + dz*dz - L1*L1 - L2*L2) / (2 * L1 * L2)
    if abs(D) > 1.0:
        raise ValueError(f"Point ({x}, {y}, {z}) is out of reach (|D|={D:.3f}>1).")

    # elbow angle:
    if elbow_up:
        theta3_rad = math.atan2(+math.sqrt(1 - D*D), D)
    else:
        theta3_rad = math.atan2(-math.sqrt(1 - D*D), D)

    # shoulder pitch:
    #    α = atan2(dz, r)
    #    β = atan2(L2*sinθ3, L1 + L2*cosθ3)
    alpha = math.atan2(dz, r)
    beta  = math.atan2(L2 * math.sin(theta3_rad), L1 + L2 * math.cos(theta3_rad))
    theta2_rad = alpha - beta

    # 4) Map back to your original angle conventions:
    #    forward did: θ2_rad = π - rad(theta2_deg)  →  theta2_deg = degrees(π - θ2_rad)
    theta2_deg = math.degrees(math.pi - theta2_rad)

    #    forward did: θ3_rad = rad(theta3_deg) - π  →  rad(theta3_deg) = θ3_rad + π
    theta3_deg = math.degrees(theta3_rad + math.pi)

    return theta1_deg, theta2_deg, theta3_deg

"""
def inverse_kinematics(x, y, z):

    #Return both elbow-down and elbow-up solutions, if reachable.

    #Returns:
        #solutions: list of (theta1_deg, theta2_deg, theta3_deg)

    sols = []
    for elbow_up in (False, True):
        try:
            sols.append(inverse_kinematics_point(x, y, z, elbow_up=elbow_up))
        except ValueError:
            # skip unreachable for that branch
            pass
    if not sols:
        raise ValueError(f"No IK solutions for point ({x}, {y}, {z}).")
    return sols

"""

'\ndef inverse_kinematics(x, y, z):\n\n    #Return both elbow-down and elbow-up solutions, if reachable.\n\n    #Returns:\n        #solutions: list of (theta1_deg, theta2_deg, theta3_deg)\n\n    sols = []\n    for elbow_up in (False, True):\n        try:\n            sols.append(inverse_kinematics_point(x, y, z, elbow_up=elbow_up))\n        except ValueError:\n            # skip unreachable for that branch\n            pass\n    if not sols:\n        raise ValueError(f"No IK solutions for point ({x}, {y}, {z}).")\n    return sols\n\n'

In [ ]:
def forward_kinematics_points(theta1_deg, theta2_deg, theta3_deg):
    theta1 = math.radians(theta1_deg) - np.pi/2
    theta2 = np.pi - math.radians(theta2_deg)
    theta3 = -(np.pi - math.radians(theta3_deg))

    l1 = 6.1   # Shoulder to elbow
    l2 = 7.0   # Elbow to end effector
    h = 6.8    # Shoulder height from ground

    x0, y0, z0 = 0, 0, 0
    x1, y1, z1 = 0, 0, h

    x2 = x1 + l1 * math.cos(theta2) * math.cos(theta1)
    y2 = y1 + l1 * math.cos(theta2) * math.sin(theta1)
    z2 = z1 + l1 * math.sin(theta2)

    total_angle = theta2 + theta3
    x3 = x2 + l2 * math.cos(total_angle) * math.cos(theta1)
    y3 = y2 + l2 * math.cos(total_angle) * math.sin(theta1)
    z3 = z2 + l2 * math.sin(total_angle)

    return [(x0, y0, z0), (x1, y1, z1), (x2, y2, z2), (x3, y3, z3)]

In [36]:
point = np.array([4.5, 5, 2])
result = inverse_kinematics_point(point[0], point[1], point[2], elbow_up=False)

fk = forward_kinematics_points(result[0], result[1], result[2])
compare = fk[3]
new_diff = np.round(point - compare, 1)

print(new_diff)

[-0.  0.  0.]


In [37]:
import itertools

range1 = range(-4, 5)   # -5 to 5
range2 = range(6, 15)   # 6 to 15
range3 = range(0, 7)    # 0 to 6

combinations = list(itertools.product(range1, range2, range3))

space = np.array(combinations)

In [38]:
diffs = np.array([0,0,0])

for point in space:
    result = inverse_kinematics_point(point[0], point[1], point[2], elbow_up=False)
    fk = forward_kinematics_points(result[0], result[1], result[2])
    compare = fk[3]
    new_diff = np.round(np.abs(point - compare), 3)
    
    diffs = np.append(diffs,new_diff)

ValueError: Point (-4, 13, 0) is out of reach (|D|=1.029>1).

In [ ]:
print(diffs)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.